In [1]:
from joblib import dump, load
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

In [2]:

clf = load('Ovsrestrfclassifier.joblib')
vectorizer_tfidf = load("TFIDF.joblib")
lb = load("Labelencoder.joblib")

In [3]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [4]:
def proba_strong(y_prend):
    y_tmp = []
    line = y_prend[0]
    for x in range(len(y_prend[0])):
        max_num = 0
        max_it = 0 
        if line[x] >= 0.10:
            y_tmp.append(x)
        if line[x] > max_num:
            max_it = x
    if y_tmp == []:
        y_tmp.append(x) 
        
    return(y_tmp)

In [5]:
def transformation_phrase(Data):
    """Data est de la forme [Titre,body]"""
    Data[0] =  review_to_words(Data[0])
    Data[1] = review_to_words(Data[1])
    Data.append(Data[1]+" "+Data[0]+" " +Data[0])
    Data.append(vectorizer_tfidf.transform([Data[-1]]))
    Data.append(proba_strong(clf.predict_proba(Data[-1])))
    Data.append(lb.inverse_transform(Data[-1]))
    l = ""
    for x in Data[-1]:
        l = l + str(x)
    return(x)


In [6]:
transformation_phrase(["Titre ios","Body in java"])

array(['ios'], dtype=object)